In [25]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [26]:
!pip install konlpy

In [0]:
%tensorflow_version 1.x
import time
import tensorflow as tf
import numpy as np
import csv
from konlpy.tag import Okt
import os
import gensim

In [0]:
class Word2Vec():
    
    def __init__(self):
        None

    def tokenize(self, doc):
        pos_tagger = Okt()
        return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]
    
    def read_data(self, filename):
        with open(filename, 'r',encoding='utf-8') as f:
            data = [line.split('\t') for line in f.read().splitlines()]
            data = data[1:]
        return data  
    
    def Word2vec_model(self, model_name):
        
        model = gensim.models.word2vec.Word2Vec.load(model_name)
        return model
    
    def Convert2Vec(self, model_name, doc): # Convert corpus into vectors
        #train_X_ = W2V.Convert2Vec("Word2Vec_csv_article.embedding",train_X)
        word_vec = []
        model = gensim.models.word2vec.Word2Vec.load(model_name)
        for sent in doc:
            sub = []
            for word in sent:
                if word in model.wv.vocab:
                    sub.append(model.wv[word]) # Word Vector Input
                else:
                    sub.append(np.random.uniform(-0.25,0.25,300)) # used for OOV words
            word_vec.append(sub)
        
        return word_vec
    
    def Zero_padding(self, train_batch_X, Batch_size, Maxseq_length, Vector_size):
        
        zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))
        for i in range(Batch_size):
            zero_pad[i,:np.shape(train_batch_X[i])[0],:np.shape(train_batch_X[i])[1]] = train_batch_X[i]
        return zero_pad
    
    def One_hot(self, data):
       
        index_dict = {value:index for index,value in enumerate(set(data))}
        result = []
        
        for value in data:
            
            one_hot = np.zeros(len(index_dict))
            index = index_dict[value]
            one_hot[index] = 1
            result.append(one_hot)
        
        return result


In [0]:
class Bi_LSTM():
    
    def __init__(self, lstm_units, num_class, keep_prob):
        
        self.lstm_units = lstm_units
        
        with tf.variable_scope('forward', reuse = tf.AUTO_REUSE):
            
            self.lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
            
        with tf.variable_scope('backward', reuse = tf.AUTO_REUSE):
            
            self.lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
        
        with tf.variable_scope('Weights', reuse = tf.AUTO_REUSE):
           
            self.W = tf.get_variable(name="W", shape=[2 * lstm_units, num_class],
                                dtype=tf.float32, initializer = tf.contrib.layers.xavier_initializer())
            self.b = tf.get_variable(name="b", shape=[num_class], dtype=tf.float32,
                                initializer=tf.zeros_initializer())
            
            
    def logits(self, X, W, b, seq_len):
        
        (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(self.lstm_fw_cell, self.lstm_bw_cell,dtype=tf.float32,
                                                                            inputs = X, sequence_length = seq_len)
        ## concat fw, bw final states
        outputs = tf.concat([states[0][1], states[1][1]], axis=1)
        pred = tf.matmul(outputs, W) + b        
        return pred
        
    def model_build(self, logits, labels, learning_rate = 0.001):
        
        with tf.variable_scope("loss"):
            
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits , labels = labels)) # Softmax loss
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) # Adam Optimizer
            
        return loss, optimizer
    
    def graph_build(self, avg_loss, avg_acc):
        
        tf.summary.scalar('Loss', avg_loss)
        tf.summary.scalar('Accuracy', avg_acc)
        merged = tf.summary.merge_all()
        return merged


In [0]:
twitter = Okt()
W2V = Word2Vec()

In [0]:
file = open("/content/gdrive/My Drive/Colab Notebooks/GraduationProject/soccer_category/dataset_shuffled.csv", 'r', encoding='utf-8')
line = csv.reader(file)
token = []
embeddingmodel = []

In [32]:
for i in line:
    content = i[0]  # csv에서 뉴스 제목 또는 뉴스 본문 column으로 변경
    sentence = twitter.pos(i[0], norm=True, stem=True)
    temp = []
    temp_embedding = []
    all_temp = []
    for k in range(len(sentence)):
        temp_embedding.append(sentence[k][0])
        temp.append(sentence[k][0] + '/' + sentence[k][1])
    all_temp.append(temp)
    embeddingmodel.append(temp_embedding)
    category = i[1]  # csv에서 category column으로 변경
    category_number_dic = {'epl': 0, 'laliga': 1, 'bundesliga': 2, 'seria':3}
    all_temp.append(category_number_dic.get(category))
    token.append(all_temp)
    
print("토큰 처리 완료")

토큰 처리 완료


In [33]:
tokens = np.array(token)
print("token 처리 완료")
print("train_data 최신 버전인지 확인")
train_X = tokens[:, 0]
train_Y = tokens[:, 1]

token 처리 완료
train_data 최신 버전인지 확인


In [34]:
train_Y_ = W2V.One_hot(train_Y)  # Convert to One-hot
train_X_ = W2V.Convert2Vec("/content/gdrive/My Drive/Colab Notebooks/GraduationProject/soccer_category/league_category.embedding",train_X)  # import word2vec model where you have trained before

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
Batch_size = 32
Total_size = len(train_X)
Vector_size = 300
seq_length = [len(x) for x in train_X]
Maxseq_length = max(seq_length)
learning_rate = 0.001
lstm_units = 128
num_class = 4
training_epochs = 18
keep_prob = 0.75

In [36]:
X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])

BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)

with tf.variable_scope("loss", reuse = tf.AUTO_REUSE):
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)

prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

total_batch = int(Total_size / Batch_size)

print("Start training!")

modelName = "/content/gdrive/My Drive/Colab Notebooks/GraduationProject/soccer_category/BiLSTM.model"
saver = tf.train.Saver()

Start training!


In [37]:
with tf.Session() as sess:

    start_time = time.time()
    sess.run(init)
    train_writer = tf.summary.FileWriter('/content/gdrive/My Drive/Colab Notebooks/GraduationProject/soccer_category/Bidirectional_LSTM', sess.graph)
  
    for epoch in range(training_epochs):

    
        for step in range(total_batch):

            train_batch_X = train_X_[step*Batch_size : step*Batch_size+Batch_size]
            train_batch_Y = train_Y_[step*Batch_size : step*Batch_size+Batch_size]
            batch_seq_length = seq_length[step*Batch_size : step*Batch_size+Batch_size]
            
            train_batch_X = W2V.Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)
            
            sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
            # Compute average loss
            loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
    
            
            acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})

            print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))


    save_path = saver.save(sess, '/content/gdrive/My Drive/Colab Notebooks/GraduationProject/soccer_category/BiLSTM.model')

    train_writer.close()
    print('save_path',save_path)

epoch : 01 step : 0001 loss = 1.331432 accuracy= 0.406250
epoch : 01 step : 0002 loss = 1.285827 accuracy= 0.531250
epoch : 01 step : 0003 loss = 1.329900 accuracy= 0.406250
epoch : 01 step : 0004 loss = 1.314764 accuracy= 0.406250
epoch : 01 step : 0005 loss = 1.242074 accuracy= 0.562500
epoch : 01 step : 0006 loss = 1.255959 accuracy= 0.437500
epoch : 01 step : 0007 loss = 1.265390 accuracy= 0.437500
epoch : 01 step : 0008 loss = 1.135215 accuracy= 0.593750
epoch : 01 step : 0009 loss = 1.113539 accuracy= 0.625000
epoch : 01 step : 0010 loss = 1.147309 accuracy= 0.656250
epoch : 01 step : 0011 loss = 0.983553 accuracy= 0.687500
epoch : 01 step : 0012 loss = 0.946257 accuracy= 0.656250
epoch : 01 step : 0013 loss = 1.025323 accuracy= 0.656250
epoch : 01 step : 0014 loss = 0.961955 accuracy= 0.750000
epoch : 01 step : 0015 loss = 0.879164 accuracy= 0.750000
epoch : 01 step : 0016 loss = 0.798227 accuracy= 0.687500
epoch : 01 step : 0017 loss = 0.967516 accuracy= 0.718750
epoch : 01 ste